In [77]:
import pandas as pd
import numpy as np
import hvplot.pandas
from IPython.display import HTML

path = 'D:\\Projects\\forecasting-challenge-4i\\'

# 1. Data Loading

In [78]:
data = pd.read_csv(path+'data\\TFP.csv')
display(data.head(3))

,isocode,year,rtfpna
0,USA,1950,0.617148
1,USA,1951,0.629588
2,USA,1952,0.638451


In [79]:
print(f'Number of Samples: {data.shape[0]}')
print(f'Number of Features: {data.shape[1]}')

Number of Samples: 186
Number of Features: 3


# 2. EDA

In [80]:
series_aux = data.dtypes
df_aux = pd.DataFrame({
                        'Feature' : series_aux.index,
                        'Data Type' : series_aux.values
})
display(HTML('<span style="font-weight:bold">' + 'Features Data Types'\
       + '</span>'), df_aux)

,Feature,Data Type
0,isocode,object
1,year,int64
2,rtfpna,float64


In [81]:
series_aux = data.isnull().sum()
df_aux = pd.DataFrame({
                        'Feature' : series_aux.index,
                        'Data Type' : series_aux.values
})
display(HTML('<span style="font-weight:bold">' + 'Features Missing Values'\
       + '</span>'), df_aux)

,Feature,Data Type
0,isocode,0
1,year,0
2,rtfpna,0


In [82]:
display(HTML('<span style="font-weight:bold">' + 'Dataset Countries and Year Range'\
       + '</span>'))
print(f'Countries: {list(data["isocode"].unique())}')
print(f'Years: {list(data["year"].unique())}')

Countries: ['USA', 'CAN', 'MEX']
Years: [1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011]


In [83]:
data.rename(columns={'isocode' :  'country'}, inplace=True)
data['decade'] = np.where(data['year'].between(1950, 1959), 1950, data['year'])
data['decade'] = np.where(data['year'].between(1960, 1969), 1960, data['decade'])
data['decade'] = np.where(data['year'].between(1970, 1979), 1970, data['decade'])
data['decade'] = np.where(data['year'].between(1980, 1989), 1980, data['decade'])
data['decade'] = np.where(data['year'].between(1990, 1999), 1990, data['decade'])
data['decade'] = np.where(data['year'].between(2000, 2009), 2000, data['decade'])
data['decade'] = np.where(data['year'].between(2010, 2019), 2010, data['decade'])
display(HTML('<span style="font-weight:bold">' + 'Transformed Data with Decade Feature'\
       + '</span>'), data.head())

,country,year,rtfpna,decade
0,USA,1950,0.617148,1950
1,USA,1951,0.629588,1950
2,USA,1952,0.638451,1950
3,USA,1953,0.651858,1950
4,USA,1954,0.646179,1950


In [84]:
plot_tfp = data.hvplot.line(x='year', y='rtfpna', by='country',
                 ylabel='TFP', yticks=np.arange(0, 1.5, 0.2).tolist(), 
                 xticks=np.arange(1950, 2011, 5).tolist(), 
                 title='TFP from 1950-2011 of NAFTA Countries')
hvplot.save(plot_tfp, path+'\\reports\\figures\\01-tfp_by_country.png')
display(plot_tfp)

:NdOverlay   [country]
   :Curve   [year]   (rtfpna)

In [85]:
tfp_mean_decade = pd.crosstab(data.country, data.decade, values=data.rtfpna,
           aggfunc='mean').round(5)

plot_tfp_mean_decade = tfp_mean_decade.hvplot.bar(x='country', y=list(map(str, tfp_mean_decade.columns.values)),
                          color='steelblue', grid=True, xlabel='Countries', ylabel='Average TFP',
                          title='Average TFP of NAFTA Countries by Decade')
hvplot.save(plot_tfp_mean_decade, path+'\\reports\\figures\\02-tfp_mean_decade_by_country.png')
display(plot_tfp_mean_decade)

:Bars   [country,Variable]   (value)

In [86]:
data_usa = data.query('country == "USA"')
data_can = data.query('country == "CAN"')
data_mex = data.query('country == "MEX"')

# 3. Time Series Model